# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [35]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [36]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [37]:
#import problem_unittests as tests
#import utils

### method 1: 
#def create_lookup_tables(text):
    #"""
    #Create lookup tables for vocabulary
    #:param text: The text of tv scripts split into words
    #:return: A tuple of dicts (vocab_to_int, int_to_vocab)
    #"""
    # TODO: Implement Function
    # get list of words
    #words = utils.preprocess(text)
    #vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
    ## return tuple
    #return (vocab_to_int, int_to_vocab)


#"""
#DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
#"""
#tests.test_create_lookup_tables(create_lookup_tables)

In [38]:
test_text = '''
        Moe_Szyslak Moe's Tavern Where the elite meet to drink
        Bart_Simpson Eh yeah hello is Mike there Last name Rotch
        Moe_Szyslak Hold on I'll check Mike Rotch Mike Rotch Hey has anybody seen Mike Rotch lately
        Moe_Szyslak Listen you little puke One of these days I'm gonna catch you and I'm gonna carve my name on your back with an ice pick
        Moe_Szyslak Whats the matter Homer You're not your normal effervescent self
        Homer_Simpson I got my problems Moe Give me another one
        Moe_Szyslak Homer hey you should not drink to forget your problems
        Barney_Gumble Yeah you should only drink to enhance your social skills'''
test_text_split = test_text.split()

In [39]:
from collections import Counter
words = test_text_split
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
int_to_vocab = {vocab_to_int[word]: word for word in vocab_to_int}
print(len(vocab_to_int))

74


In [40]:
import problem_unittests as tests
from string import punctuation
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    #script_split = text.split('\n')
    #all_text = ' '.join(script_split)
    #words = all_text.split()
    words = text
    counts = Counter(words)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word: ii for ii, word in enumerate(vocab, 0)}
    int_to_vocab = {vocab_to_int[word]: word for word in vocab_to_int}

    #chars = tuple(set(words))
    #int_to_vocab = dict(enumerate(chars))
    #vocab_to_int = {ch: ii for ii, ch in int2char.items()}
    
    #vocab = set(text)
    #vocab_to_int = {word: ii for ii, word in enumerate(vocab)}
    #int_to_vocab = dict(enumerate(vocab))
    #return (vocab_to_int, int_to_vocab)
    
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


In [41]:
## test the implemented function:
(vocab_to_int, int_to_vocab) = create_lookup_tables(text.split())
print('Unique words: ', len(vocab_to_int))
print('Unique vocab: ', len(int_to_vocab))

Unique words:  46367
Unique vocab:  46367


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [20]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punct_to_token = {'.': '||Period||', 
        ',': '||Comma||',
        '"': '||Quotation_Mark||',
        ';': '||Semicolon||',
        '!': '||Exclamation_mark||',
        '?': '||Question_mark||',
        '(': '||Left_Parentheses||',
        ')': '||Right_Parentheses||',
        '-': '||Dash||',
        '\n': '||Return||'}   
    return punct_to_token

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to look at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [46]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [47]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

No GPU found. Please use a GPU to train your neural network.


## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [5]:
import numpy as np
def get_batches(words, sequence_length):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    ## code starts: 
    len_of_texts = len(words) 
    #batch_x = []
    #batch_y = []
    feature_batches = []
    target_batches = []
    for i in range(len_of_texts):
            ##batch * stride + item * batch_size * sequence_length, with stride = 1
            begin_x = i
            end_x = begin_x + sequence_length
            if end_x < len_of_texts - 1:
                position_y = end_x + 1
                #print("begin_x = ", begin_x)
                #print("end_x = ", end_x)
                #print("position_y = ", position_y)
                #batch_x = words[begin_x:end_x]
                #batch_y = words[position_y]
                feature_batches.append(words[begin_x:end_x])
                target_batches.append(words[position_y-1])
    return np.array(feature_batches), np.array(target_batches)

In [6]:
## small test: 
print(get_batches([1, 2, 3, 4, 5, 6, 7], 4))

(array([[1, 2, 3, 4],
       [2, 3, 4, 5]]), array([5, 6]))


In [7]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    ## get the batches contain sequence lengths of data    
    feature_tensor, target_tensor = get_batches(words, sequence_length)
    data = TensorDataset(torch.from_numpy(feature_tensor), torch.from_numpy(target_tensor))
    data_loader = torch.utils.data.DataLoader(data, shuffle=True, batch_size=batch_size)
    # return a dataloader
    return data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [8]:
# test dataloader

test_text = np.array(range(50))
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()
sample_x, sample_y = next(data_iter)

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 23,  24,  25,  26,  27],
        [  8,   9,  10,  11,  12],
        [ 42,  43,  44,  45,  46],
        [  7,   8,   9,  10,  11],
        [ 35,  36,  37,  38,  39],
        [  2,   3,   4,   5,   6],
        [ 22,  23,  24,  25,  26],
        [ 10,  11,  12,  13,  14],
        [ 14,  15,  16,  17,  18],
        [ 17,  18,  19,  20,  21]])

torch.Size([10])
tensor([ 28,  13,  47,  12,  40,   7,  27,  15,  19,  22])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

### load the pretrained embedding layer: 

In [1]:
''' load the embedding dimensions: '''
def from_pretrained(embeddings, freeze=True):
    assert embeddings.dim() == 2, \
         'Embeddings parameter is expected to be 2-dimensional'
    rows, cols = embeddings.shape
    embedding = torch.nn.Embedding(num_embeddings=rows, embedding_dim=cols)
    embedding.weight = torch.nn.Parameter(embeddings)
    embedding.weight.requires_grad = not freeze
    return embedding

#embedding = from_pretrained(weights)


In [ ]:
if train_on_gpu:
    embed.cuda()
    embed.load_state_dict(torch.load('model_embedding.pt'))
else: 
    embed.load_state_dict(torch.load('model_embedding.pt', map_location=lambda storage, loc: storage))

In [9]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.2):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.embed_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        
        # define model layers
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=dropout, batch_first=True)
        
        ## dropout layer
        #self.dropout = nn.Dropout(0.3)
        
        ## linear and softmax layers
        self.fc = nn.Linear(hidden_dim, output_size)
        ##self.soft_max = nn.softmax(dim=1)
        
        ## Initialize embedding tables with uniform distribution
        ## here embedding layer only used as dimension reduction tool
        ## rather than look up tables. 
        # self.embedding.weight.data.uniform_(-1, 1)
        ## or load the embedding layer from pretrained model
        #embed_tensor is a torch tensor.
        #embedding = nn.Embedding(embed_tensor.size(0), embed_tensor.size(1))
        #embedding.weight = nn.Parameter(embed_tensor)
        ## method 2: 
        #embed = nn.Embedding(num_embeddings, embedding_dim)
        ## pretrained_weight is a numpy matrix of shape (num_embeddings, embedding_dim)
        #embed.weight.data.copy_(torch.from_numpy(pretrained_weight))
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        
        batch_size = nn_input.size(0)
        # embeddings and lstm_out
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        #lstm_out = self.dropout(lstm_out)
        out = self.fc(lstm_out)
        # softmax function
        #soft_out = self.soft_max(out)
        ## try without the softmax function
        soft_out = out
        
        # reshape to be batch_size first
        #soft_out = soft_out.view(batch_size, -1)
        # reshape into (batch_size, seq_length, output_size)
        soft_out = soft_out.view(batch_size, -1, self.output_size)
        # get the last batch
        soft_out = soft_out[:, -1] # get last batch of labels
        
        # return one batch of output word scores and the hidden state
        # return last softmax output and hidden state
        return soft_out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # initialize hidden state with zero weights, and move to GPU if available
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


In [10]:
## test: 

### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [11]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder = rnn: The PyTorch Module that holds the neural network
    :param decoder_optimizer = optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp = parameters: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    # move data to GPU, if available
    if(train_on_gpu):
        rnn.cuda()
        inp, target = inp.cuda(), target.cuda()
    
    rnn.train()
    
    hidden = tuple([each.data for each in hidden])

    # zero accumulated gradients
    rnn.zero_grad()

    # get the output from the model
    output, hidden = rnn(inp, hidden)
    
    # perform backpropagation and optimization
    # calculate the loss and perform backprop
    loss = criterion(output.squeeze(), target.long())
    loss.backward()
    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
    clip=5 # gradient clipping
    nn.utils.clip_grad_norm_(rnn.parameters(), clip)
    optimizer.step()

    ## return the loss over a batch (average loss in next block of code) 
    ## and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    ## earlier stopping
    loss_min = np.Inf 
    stop_counter = 0
    ## recored losses for plot
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        ## initialize the counter for earlier stopping
        stop_counter = 0
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        ### adaptive learning rate`
        ### decrease lr every 10 epochs
        if epoch_i % 5 == 0:
            print("learning rate in epoch", epoch_i, "before change:", optimizer.param_groups[0]["lr"])
            lr = init_lr * (0.1 ** (epoch_i // 5))
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr 
            print("after change:", optimizer.param_groups[0]["lr"])
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                
                avg_batch_losses = np.average(batch_losses)                
                ## earlier stopping
                ## TODO: save the model if validation loss has decreased
                if avg_batch_losses <= loss_min:
                    print('avg batch loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                    loss_min,
                    avg_batch_losses))
                    ## 'model_transfer.pt' = save_path
                    torch.save(rnn.state_dict(), 'model_rnn.pt')
                    loss_min = avg_batch_losses
                    stop_counter = 0
                else: 
                    print('avg batch loss increased ({:.6f} --> {:.6f}).  not saving model ...'.format(
                    loss_min,
                    avg_batch_losses))
                    ## count the number of increased loop
                    stop_counter += 1
                    if(stop_counter > 15):
                        helper.save_model('./save/trained_rnn', trained_rnn)
                        assert stop_counter==16, "avg batch loss stopped decreasing over 10 steps."
                        stop_counter = 0
                        #print("batch loss stopped decreasing over 5 steps")
                        #break
                
                batch_losses = []
                
                
                ## change the name, for saving multiple files
                ## try to implement it into train block cell, and save model state when loss 
                ## is decreasing within 10 x show_every_n_batches steps
                #model_name = 'rnn_tv.net'
                        ##set class variables
                    #    self.output_size = output_size
                    #    self.n_layers = n_layers
                     #   self.embed_dim = embedding_dim
                     #   self.hidden_dim = hidden_dim
                      #  self.vocab_size = vocab_size
                #checkpoint = {'n_hidden': rnn.hidden_dim,
                             # 'output_size': rnn.output_size,
                             # 'embedding_dim': rnn.embedding_dim,
                            #  'n_layers': rnn.n_layers,
                            #  'state_dict': rnn.state_dict(),
                            #  'vocab_size': rnn.vocab_size}

                #with open(model_name, 'wb') as f:
                    #torch.save(checkpoint, f)
                #######################################   

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [13]:
# Data params
# Sequence Length
sequence_length = 30  # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [14]:
# Training parameters
# Number of Epochs
num_epochs = 50
# Learning Rate, 
learning_rate = 0.002
init_lr = learning_rate
# Model parameters
# Vocab size
vocab_size = len(int_to_vocab)
# Output size
output_size = len(int_to_vocab)
# Embedding Dimension 213,
embedding_dim = 463 
# Hidden Dimension
## the hidden layer
hidden_dim = 512
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
### keep the cell running longer than 30 mins with GPU 
import workspace_utils
from workspace_utils import active_session

with active_session():
    
    # create model and move to gpu if available
    rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.3)
    if train_on_gpu:
        rnn.cuda()

    # defining loss and optimization functions for training
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    helper.save_model('./save/trained_rnn', trained_rnn)
    print('Model Trained and Saved')

Training for 50 epoch(s)...
Epoch:    1/50    Loss: 5.133680098056793

avg batch loss decreased (inf --> 5.133680).  Saving model ...
Epoch:    1/50    Loss: 4.558099571228027

avg batch loss decreased (5.133680 --> 4.558100).  Saving model ...
Epoch:    1/50    Loss: 4.394286631584167

avg batch loss decreased (4.558100 --> 4.394287).  Saving model ...
Epoch:    1/50    Loss: 4.305378020763397

avg batch loss decreased (4.394287 --> 4.305378).  Saving model ...
Epoch:    1/50    Loss: 4.259002668380737

avg batch loss decreased (4.305378 --> 4.259003).  Saving model ...
Epoch:    1/50    Loss: 4.203367830276489

avg batch loss decreased (4.259003 --> 4.203368).  Saving model ...
Epoch:    1/50    Loss: 4.151672180652619

avg batch loss decreased (4.203368 --> 4.151672).  Saving model ...
Epoch:    1/50    Loss: 4.129085820674896

avg batch loss decreased (4.151672 --> 4.129086).  Saving model ...
Epoch:    1/50    Loss: 4.1000419540405275

avg batch loss decreased (4.129086 --> 4.1000

KeyboardInterrupt: 

In [ ]:
# change the name, for saving multiple files
# try to implement it into train block cell, and save model state when loss 
# is decreasing within 10 x show_every_n_batches steps
#model_name = 'rnn_tv.net'
#        # set class variables
#        self.output_size = output_size
#        self.n_layers = n_layers
#        self.embed_dim = embedding_dim
#        self.hidden_dim = hidden_dim
#        self.vocab_size = vocab_size
#checkpoint = {'n_hidden': rnn.hidden_dim,
#              'output_size': rnn.output_size,
#              'embedding_dim': rnn.embedding_dim,
#              'n_layers': rnn.n_layers,
#              'state_dict': rnn.state_dict(),
#              'vocab_size': rnn.vocab_size}
#
#with open(model_name, 'wb') as f:
#    torch.save(checkpoint, f)

In [15]:
## loading model: 
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.3)
if train_on_gpu:
    rnn.cuda()
    rnn.load_state_dict(torch.load('model_rnn.pt'))
else: 
    rnn.load_state_dict(torch.load('model_rnn.pt', map_location=lambda storage, loc: storage))

In [17]:
def parameter_counter(model):
    return sum(c.numel() for c in model.parameters() if c.requires_grad)

count = parameter_counter(rnn)
print("the number of total trainable parameters in model is: ", count)

the number of total trainable parameters in model is:  24976832


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

## **Answer:** 
### Learning rate: 
Learing rate can be 0.1, 0.01, 0.0001, 0.00001, etc. Alternatively, apaptive learning rate can be applied as: 

`### adaptive learning rate
if(e > 20):
    print("learning rate in epoch", e, "before change:", optimizer.param_groups[0]["lr"])
    lr = init_lr * (0.1 ** (e // 20))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr 
    print("after change:", optimizer.param_groups[0]["lr"])`
When training a model, it is often recommended to lower the learning rate as the training progresses. The learning rate can be adjusted as learning rate linear decay, or exponentially decay. 

### dropout rate: 
the dropout layer is used to mitigates the overfitting. It can be number between 0 - 1, in general, we can compare the training loss and validation loss if they are further apart during training process (training loss much smaller than val loss), it indicates overfitting of the model. In other words, the model is trying to memorize the data instead of learning it. 
The dropout rate is 0.3 which is optimal in our case. 

### batches: 
The good candidate for batch sizes are: $$1, 2, 4, 8, 16, 32, \cdots 256.$$ 
The mini batchsize, too small the training process is too slow, too large cause too much memory (computationaly taxing) and decreasing of validation and test accuracy. The batch size between $32 - 256$ (range) would be good starting point, in order to test the mini batch size values. (Systematic evaluation of CNN advances on the ImageNet, Mishkin et al, 2016) 


Recommendation from Udacity: setting batch_size as a power of 2 (128 in your case) is handled efficiently by torch (better so on GPU).




### epochs:
Regarding to the epochs, we can set stops the training process once the validation error is stopped decreasing, or it stopped decreasing after 10 to 20 steps, then we call stop the iterations. The technique is called **earlier stopping**. The number of epochs essentially depends on the trend of validation errors. The earlier stopping can be easily implemented manually, or it is built in with Tensorflow, but not Pytorch. 

I stopped the iteration earlier since I don't have much GPU time left, but in practice, I would expect my final loss to be around 1.0 after 50 epochs.  
### hidden layers:
The increased hidden layers tends to overfitting the data, which appears as departs of validation loss from training loss. However, we can fix this by adopting regularization techniques such as: Dropout or L2 regularazation, or more data sets. In practice, keep adding the hidden units until the validation loss are getting worse (stopped decreased or increased). In general the number of hidden units can be larger than the number of inputs. 
2 - 3 would be enough (Long Short-Term Memory Recurrent Neural Network Architectures
for Large Scale Acoustic Modeling)

### hidden dimensions: 
The hidden_dim usually takes common values between 128, 256, 512, etc. We take 512 in this case. 


### embedding dimensions: 
The number of embedding_dim and hidden layers can prevent underfitting, the more the layers the better the accuracy. The typical embedding dimension would be between 200–300 (source: Google). The embedding dimension is set to match the size of the total unique vocab size, which is $1\%$ of the size of vocab.

### sequence lengths: 
The SL impacts on the datasets's long term dependencies. 30 should be suffice to include a sentence of a person said in TV plot script. I find the longer the sequence length, the faster the convergence speed, but once the sequence length exceeds 30, the initial loss starts to increase.  

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

#_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
#trained_rnn = helper.load_model('./save/trained_rnn')

## loading model: 
_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
#rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()
    rnn.load_state_dict(torch.load('model_rnn.pt'))
else: 
    rnn.load_state_dict(torch.load('model_rnn.pt', map_location=lambda storage, loc: storage))

In [22]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
#with open('rnn_tv.net', 'rb') as f:
    #checkpoint = torch.load(f)
    
#loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
#loaded.load_state_dict(checkpoint['state_dict'])

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [30]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'george' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
#generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
generated_script = generate(rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

george: ba, to figuring out of your life....

jerry: well, he doesn't even know what he's talking about.

george: oh my goodness.

jerry: you know what? maybe i can go to a funeral.

kramer: oh, no no no no no. no, i don't think we should go.

george: i know...

jerry: i don't know.

kramer:(to george) what are you doing?

kramer: you know, i don't even have any idea who i was going to do.

jerry: what about your schedule?

kramer: well, i just want to go to the movies.

jerry: oh, i think i got a deal.

elaine:(still laughing, opens door) i know..

jerry:(to the tv) what is wrong with you?

george: well, i was just thinking about that statue.

jerry: i can't believe it.

elaine: what?

george: i think you forgot your bitter.

george: i don't understand.

jerry: oh yeah? well, i guess i was thinking about that. i don't have any trouble finding the bench.

kramer: oh, no.

newman: well, the only reason we have to tell you what i think!

jerry: what about a letter?

elaine: no, i can't b

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [31]:
# save script to a text file
f =  open("generated_script_2.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.

##### discard uncommon words:
Use negative sampling